# Pull request scraper

## GitHub credentials
A private access token is necessary to make use of less restrictive API limits.

In [1]:
from github import RateLimitExceededException, Github

# Providing access token
access_token = ""
g = Github(login_or_token=access_token)

# Confirm your login is successful
user = g.get_user()
print(f"Authenticated as: {user.login}")

Request GET /user failed with 403: Forbidden
Setting next backoff to 82.925177s


Authenticated as: AbelvdTil


## Files

In [107]:
import os
 
STEP4_TFCOMMITS = os.path.join("data", "previous-study", "step4-tf-commits.json") 
COMMIT_LABELS = os.path.join("data", "process-labeled-commits", "full-commit-labels.json") 
NEW_COMMITS = os.path.join("data", "update-previous-study", "new-commits-dataset.json") 
OLD_COMMITS = os.path.join("data", "previous-study", "dataset.json") 

STEP5_TF_REPOS_WITH_PR = os.path.join("data", "pullrequest-scraping", "step5-tf-repos-with-pr.json")
STEP6_TF_REPOS_COMMITS = os.path.join("data", "pullrequest-scraping", "step6-tf-repos-commits.json")
STEP6A_TF_REPOS_RELEVANT_COMMITS = os.path.join("data", "pullrequest-scraping", "step6a-tf-repos-relevant-commits.json")
STEP7_TF_REPOS_WITH_TF_PR = os.path.join("data", "pullrequest-scraping", "step7-tf-repos-with-tf-pr.json")
STEP8_TF_KEYWORD_PR = os.path.join("data", "pullrequest-scraping", "step8-tf-keyword-pr.json")
STEP9_TF_PR_DATASET = os.path.join("data", "pullrequest-scraping", "step9-tf-pr-dataset.json")
STEP11_RELEVANT_DATASET = os.path.join("data", "pullrequest-scraping", "step11-relevant-tf-pr-dataset.json")
COMMIT_PR_INTERSECTION = os.path.join("data", "pullrequest-scraping", "commit-pr-intersection.json")

## File seperator and combinator

Files over 100MB are not stored on GitHub, therefore we need to seperate large files into smaller ones.
Any step5 and step7 files can be split into multiple smaller ones and be combined together.

The repositories are split on pull request level. The first pull request is stored in part 1, the 2nd in part 2 etc. This will make sure that it is fairly equally distributed.

Each pull request is accomodated with the url of the repository. Therefore it is possible to reconstruct the original file again.

In [4]:
import json
import math

FILE_TO_SEPERATE = STEP5_TF_REPOS_WITH_PR

nr_parts = 10

### File seperator

In [46]:
parts_data = []

for i in range(nr_parts):
    parts_data.append([])

file = open(FILE_TO_SEPERATE)
seperator_data = json.load(file)

count = 0
for repository in seperator_data:
    for pull_request in repository["pull_requests"]:
        part = (count % nr_parts)
        count += 1
        parts_data[part].append({"repo_url": repository["url"], "pull_request": pull_request})

for i in range(nr_parts):
    with open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(i+1) + ".json", "w") as outfile:
        json.dump(parts_data[i], outfile)

### File combinator

In [5]:
url_dict = {}
combinator_data = []
index_count = 0
for part in range(nr_parts):
    part_file = open(FILE_TO_SEPERATE.split(".")[0] + "-part-" + str(part+1) + ".json", "r")
    part_data = json.load(part_file)

    for nugget in part_data:
        # find repo using url
        index = url_dict.get(nugget["repo_url"], None)

        if (index == None):
            repo = {"url": nugget["repo_url"], "pull_requests": []}
            combinator_data.append(repo)
            
            url_dict[nugget["repo_url"]] = index_count
            index_count += 1
        else:
            repo = combinator_data[index]
            
        repo["pull_requests"].append(nugget["pull_request"])

with open(FILE_TO_SEPERATE, "w") as outfile:
    json.dump(combinator_data, outfile) 

## STEP 1: Pull request scraping script

For each repository, get all pull request data. This includes PR Title, description, (review) comments and commit hashes. Exclude any repositories that do not have pull requests.

In [7]:
# SETTINGS
check_limit_every_x_calls = 5
api_limit_buffer = 10
api_calls_per_debug = 500

# INITIALIZATION

import json
import datetime
import time

# INITIALIZATION
terraform_output = open(STEP4_TFCOMMITS)
step4_output = json.load(terraform_output)

# Retrieve data from previous run
try:
    previous_run = open(STEP5_TF_REPOS_WITH_PR)
    repoData_dict = json.load(previous_run)
except FileNotFoundError as e:
    repoData_dict = []

iteration = 0
calls_till_next_debug = 0
calls_till_limit_checkup = 0

# Check for api limits, also periodically calls print debug.
def CheckForApiLimit():
    global calls_till_limit_checkup
    global calls_till_next_debug
    global api_calls_per_debug
    global api_limit_buffer

    # check for limit
    if (calls_till_limit_checkup == 0):
        core_limit = g.get_rate_limit().core

        # sleep when exceeded api core limit
        if (core_limit.remaining <= api_limit_buffer):
            time_to_sleep = core_limit.raw_data['reset'] - time.time() + 1
            print("Rate limit exceeded, sleeping for", time_to_sleep, "seconds.", "Actual remaining calls", core_limit.remaining)
            time.sleep(time_to_sleep)

        calls_till_limit_checkup = check_limit_every_x_calls
    
    calls_till_limit_checkup -= 1

    # check for debug
    if (calls_till_next_debug == 0):
        PrintDebug()
        calls_till_next_debug = api_calls_per_debug

    calls_till_next_debug -= 1

# Prints debug message
def PrintDebug():
    global iteration
    global repo_url

    print(datetime.datetime.now().strftime("%H:%M:%S"), ":", 
              "current iteration:", iteration, 
              "url:", repo_url)

# Pull request scraping script
for rp in step4_output["repositories"]:
    try:
        iteration += 1
            
        repo_url = rp["name"]

        # skip already scraped repositories
        if any(d["url"] == repo_url for d in repoData_dict):
            continue

        # Get the repo object from the url
        split_list = repo_url.split("/")
        actual_url = (split_list[3]+ '/' + split_list[4]).split('.git')[0]
        repo = g.get_repo(actual_url)
        
        # Get required info for pull requests
        pull_requests_dict = []
        pull_requests = repo.get_pulls(state="closed")

        if pull_requests.totalCount > 0:
            for pr in pull_requests:
    
                # retrieve all review comments, not required if there are none.
                comments = []
            
                for review in pr.get_reviews():
                    if (review.body.strip() != ""):
                        comments.append(review.body)
                CheckForApiLimit()

                if (pr.review_comments > 0):
                    for review_comment in pr.get_review_comments():
                        if (review_comment.body.strip() != ""):
                            comments.append(review_comment.body)
                    CheckForApiLimit()

                if (pr.comments > 0):
                    for comment in pr.get_issue_comments():
                        if (comment.body.strip() != ""):
                            comments.append(comment.body)
                    CheckForApiLimit()
    
                # retrieve all connected commits.
                commits = []
                for commit in pr.get_commits():
                    commits.append(commit.sha)
                CheckForApiLimit()
    
                pull_requests_dict.append({"url": pr.html_url, "title": pr.title, "body": pr.body, "comments": comments, "commits": commits})
            
            CheckForApiLimit()        
            repoData_dict.append({"url": repo_url, "pull_requests": pull_requests_dict});
            
            with open(STEP5_TF_REPOS_WITH_PR, "w") as outfile:
                json.dump(repoData_dict, outfile)
    except Exception as e:
        print("exception:", e)

exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message

Following Github server redirection from /repos/andreas-prinz/gcp-terraform-google-lb to /repositories/425986937


exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}


Following Github server redirection from /repos/lwilliams1990/deepfence-threatmapper-lab to /repositories/270369845


exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message

...

exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}


Following Github server redirection from /repos/monish-advani/terragoat-test to /repositories/304530642


exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message

Following Github server redirection from /repos/dgorbov/terraform-s3-backend-setup to /repositories/230207730


exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}
exception: 404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest/repos/repos#get-a-repository"}


## STEP 2: Find tf commits for tf repos with pr's 

For remaining repositories from step 5, collect all commits that modify a terraform file.

In [14]:
from pydriller import Repository

import json

step5 = open(STEP5_TF_REPOS_WITH_PR)
tf_repositories = json.load(step5)

terraform_keywords = ['.tf', '.tf.json']

iteration = 0
    
# Pull request scraping script
repo_dic = []
for repository in tf_repositories:
    try:
        if (iteration % 50 == 0):
            print("at iteration", iteration)
            with open(STEP6_TF_REPOS_COMMITS, "w") as outfile:
                json.dump(repo_dic, outfile)

        iteration += 1

        # Get each commit
        commit_dic = []
        for commit in Repository(repository["url"]).traverse_commits():

            modified_terraform = False
            # find if it changes a terraform file
            for file in commit.modified_files:
                if any(key in file.filename for key in terraform_keywords):
                    modified_terraform = True
            
            if modified_terraform:
                commit_dic.append({"hash": commit.hash, 
                                   "url": repository["url"].split(".git")[0] + "/commit/" + commit.hash, 
                                   "date": str(commit.author_date), 
                                   "body": commit.msg})
  
        repo_dic.append({"url":repository["url"], "commits":commit_dic})
        
    except Exception as e:
        print("exception:", e)

with open(STEP6_TF_REPOS_COMMITS, "w") as outfile:
        json.dump(repo_dic, outfile)

at iteration 0
at iteration 50
at iteration 100
at iteration 150
at iteration 200
at iteration 250
at iteration 300
at iteration 350
at iteration 400
at iteration 450
at iteration 500
at iteration 550
at iteration 600


## STEP 3: exclude unrelated tf commits

In [16]:
import json

step6 = open(STEP6_TF_REPOS_COMMITS)
repository_commits = json.load(step6)

labels_file = open(COMMIT_LABELS)
commit_labels = json.load(labels_file)

for repository in repository_commits:
    commits = []
    for commit in repository["commits"]:
        label = commit_labels.get(commit["hash"], None)
        
        if label is None or "unrelated" not in label:
            commits.append(commit)
    repository["commits"] = commits

with open(STEP6A_TF_REPOS_RELEVANT_COMMITS, "w") as outfile:
        json.dump(repository_commits, outfile)

## STEP 4: filter out pull requests without relevant tf commit

Removes any pull request that does not include a commit from the previous step, for the remaining pull requests, it combines the two datasets into one.

In [41]:
import json

step5 = open(STEP5_TF_REPOS_WITH_PR)
repository_input = json.load(step5)

step6a = open(STEP6A_TF_REPOS_RELEVANT_COMMITS)
commit_input = json.load(step6a)

iteration = 0

output_dict = []

# for each repository
for repository in repository_input:
    # find commits for repo from step 6a
    commit_input_list = next(repo["commits"] for repo in commit_input if repo["url"] == repository["url"])

    pr_dict = []
    # for each pull request
    for pull_request in repository["pull_requests"]:
        commit_dict = []

        # for each commit
        for commit_hash in pull_request["commits"]:
            # Find the exact commit from step 7
            commit_data = next((commit for commit in commit_input_list if commit["hash"] == commit_hash), None)
            if (commit_data is not None):
                commit_dict.append(commit_data)

        pull_request["total_commits"] = len(pull_request["commits"])
        pull_request["commits"] = commit_dict
        
        if (len(commit_dict) > 0):
            pr_dict.append(pull_request)
    
    if (len(pr_dict) > 0):
        output_dict.append({"url": repository["url"], "pull_requests": pr_dict})

with open(STEP7_TF_REPOS_WITH_TF_PR, "w") as outfile:
    json.dump(output_dict, outfile) 

## STEP 5: list all tf pull request with a keyword

In [42]:
cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step7 = open(STEP7_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step7)

pullrequest_dict_output = []
for repository in repo_input:
    for pr in repository["pull_requests"]:
        
        title   = True if (pr["title"]        is not None and any(key in pr["title"].lower()    for key in cost_keywords)) else False
        body    = True if (pr["body"]         is not None and any(key in pr["body"].lower()     for key in cost_keywords)) else False
        comment = True if (any(comment        is not None and     key in comment.lower()        for key in cost_keywords for comment in pr["comments"])) else False
        commit  = True if (any(commit["body"] is not None and     key in commit["body"].lower() for key in cost_keywords for commit  in pr["commits"]))  else False
            
        reason = (("title " if title else "") + 
                  ("body " if body else "") + 
                  ("comment " if comment else "") + 
                  ("commit " if commit else ""))
        
        if (title or body or comment or commit):
            pullrequest_dict_output.append({"reason": reason.strip(), "pull_request": pr})

with open(STEP8_TF_KEYWORD_PR, "w") as outfile:
    json.dump(pullrequest_dict_output, outfile) 

## Intermediate results

In [43]:
import json

cost_keywords = ["cheap", "expens", "cost", "efficient", "bill", "pay"]

step4 = open(STEP4_TFCOMMITS)
step4_data = json.load(step4)

step5 = open(STEP5_TF_REPOS_WITH_PR)
step5_data = json.load(step5)

step7 = open(STEP7_TF_REPOS_WITH_TF_PR)
repo_input = json.load(step7)

step8 = open(STEP8_TF_KEYWORD_PR)
pr_reason_input = json.load(step8)


# GENERAL REPOSITORY DATA

print("Total repositories:" , step4_data["no_of_repos"])
print("Of those that exist and have pull request(s):", len(step5_data))
print("Of those that have relevant TF commits:", len(repo_input))

print("")

# GENERAL PULL REQUEST DATA
print("Total TF PR's with a keyword:", len(pr_reason_input))

print("")

print("PR with keyword in:\t", "Only in:")
print("Title:\t\t", len([pr for pr in pr_reason_input if "title" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "title" == pr["reason"]]))
print("Description:\t", len([pr for pr in pr_reason_input if "body" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "body" == pr["reason"]]))
print("Comment:\t", len([pr for pr in pr_reason_input if "comment" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "comment" == pr["reason"]]))
print("commit message*:", len([pr for pr in pr_reason_input if "commit" in pr["reason"]]), "\t", len([pr for pr in pr_reason_input if "commit" == pr["reason"]]))  

print("")

print("*commits labeled as unrelevant have already been removed from the dataset, while the same is not true for the other locations.")

print("")

# GENERAL COMMIT DATA

print("Total amount of relevant commits in TF PR's:", sum([pr["total_commits"] for repo in repo_input for pr in repo["pull_requests"]]))
print("Of those that modify a TF file:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"]]))
print("Of those that have a keyword:", len([commit for pr_reason in pr_reason_input for commit in pr_reason["pull_request"]["commits"] if any(key in commit["body"].lower() for key in cost_keywords)]))

print("")

count = 0
for repo in repo_input:
    for pr in repo["pull_requests"]:
        if (pr["total_commits"] >= 250):
            count += 1
print("Amount of PR's with more than 250 commits (limit):", count)
    


Total repositories: 1278
Of those that exist and have pull request(s): 610
Of those that have relevant TF commits: 469

Total TF PR's with a keyword: 814

PR with keyword in:	 Only in:
Title:		 111 	 30
Description:	 363 	 214
Comment:	 354 	 322
commit message*: 194 	 63

*commits labeled as unrelevant have already been removed from the dataset, while the same is not true for the other locations.

Total amount of relevant commits in TF PR's: 130720
Of those that modify a TF file: 3266
Of those that have a keyword: 203

Amount of PR's with more than 250 commits (limit): 34


## STEP 6: Parse to dataset format

In [44]:
import json

step8 = open(STEP8_TF_KEYWORD_PR)
pull_request_reasons = json.load(step8)

pr_output = []
for pull_request_reason in pull_request_reasons:
    pull_request = pull_request_reason["pull_request"]
    commits = []
    for commit in pull_request["commits"]:
        commits.append(commit["hash"])
    pr_output.append(
        {
            "type": "pull_request", 
            "url": pull_request["url"],
            "content": {
                "title": pull_request["title"],
                "body": pull_request["body"],
                "comments": pull_request["comments"],
                "commits": commits
                },
            "codes": []
        })

with open(STEP9_TF_PR_DATASET, "w") as outfile:
    json.dump(pr_output, outfile) 

## STEP 7: Manually label dataset

This cannot be done through code

## STEP 8: Filter out unrelated pull requests

In [2]:
import json
import os
   
data_file = open(STEP9_TF_PR_DATASET)
data = json.load(data_file)

output = []
for pr in data:
    if pr["codes"] != ["unrelated"]:
        output.append(pr)

with open(STEP11_RELEVANT_DATASET , "w") as outfile:
    json.dump(output, outfile)

## STEP 12: label results

In [105]:
import json

# retrieve and simplify pr dataset
all_pull_requests_file = open(STEP9_TF_PR_DATASET)
all_pull_requests = json.load(all_pull_requests_file)

for pull_request in all_pull_requests:
    labels = []

    for label in pull_request["codes"]:
        if "feature" in label:
            labels.append("feature")
        elif "policy" in label:
            labels.append("policy")
        elif "area" in label:
            labels.append("area")
        else:
            labels.append(label)

    labels.sort()
    pull_request["codes"] = labels

# retrieve and simplify pr dataset
pull_requests_file = open(STEP11_RELEVANT_DATASET)
pull_requests = json.load(pull_requests_file)

for pull_request in pull_requests:
    labels = []

    for label in pull_request["codes"]:
        if "feature" in label:
            labels.append("feature")
        elif "policy" in label:
            labels.append("policy")
        elif "area" in label:
            labels.append("area")
        else:
            labels.append(label)

    labels.sort()
    pull_request["codes"] = labels

# retrieve and simplify old commit dataset
old_commits_file = open(OLD_COMMITS)
old_commits = json.load(old_commits_file)
old_commits = [commit for commit in old_commits if commit["type"] == "commit"]

# retrieve and simplify and append new commit dataset
all_commits_file = open(NEW_COMMITS)
all_commits = json.load(all_commits_file)
all_commits = [commit for commit in all_commits if commit["codes"] != ["unrelated"]]
all_commits = old_commits + all_commits 

### Label histogram

In [87]:
label_hist = {}

for nugget in pull_requests:
    for label in nugget["codes"]:
        if label_hist.get(label, None) is None:
            label_hist[label] = {"prs": 0, "old_commits": 0, "all_commits": 0}

        label_hist[label]["prs"] = label_hist[label]["prs"] + 1

for nugget in old_commits:
    for label in nugget["codes"]:
        if label_hist.get(label, None) is None:
            label_hist[label] = {"prs": 0, "old_commits": 0, "all_commits": 0}

        label_hist[label]["old_commits"] = label_hist[label]["old_commits"] + 1

for nugget in all_commits:
    for label in nugget["codes"]:
        if label_hist.get(label, None) is None:
            label_hist[label] = {"prs": 0, "old_commits": 0, "all_commits": 0}

        label_hist[label]["all_commits"] = label_hist[label]["all_commits"] + 1


header = ["label", "old (%)", "all (%)", "prs (%)"]
print('{:20} |  {:9} {:9} {:9}'.format(*header)) 
print()
for label in label_hist:
    row = [label, 
           label_hist[label]["old_commits"], "(" + str(int(label_hist[label]["old_commits"] / len(old_commits) * 100)) + ")",
           label_hist[label]["all_commits"], "(" + str(int(label_hist[label]["all_commits"] / len(all_commits) * 100)) + ")",
           label_hist[label]["prs"], "(" + str(int(label_hist[label]["prs"] / len(pull_requests) * 100)) + ")"
          ]
    print('{:20} | {:4} {:4} {:4} {:4} {:4} {:4}'.format(*row)) 

label                |  old (%)   all (%)   prs (%)  

instance             |  154 (28)  170 (28)   79 (31)
saving               |  366 (68)  404 (66)  125 (49)
awareness            |  203 (37)  213 (35)   87 (34)
storage              |   72 (13)   82 (13)   62 (24)
alert                |   45 (8)    59 (9)    33 (13)
feature              |   51 (9)    67 (11)   67 (26)
increase             |   12 (2)    18 (2)    21 (8) 
cluster              |   13 (2)    17 (2)    27 (10)
networking           |   52 (9)    55 (9)    20 (7) 
provider             |   25 (4)    25 (4)     7 (2) 
billing_mode         |   29 (5)    32 (5)     9 (3) 
policy               |   11 (2)    16 (2)     6 (2) 
area                 |   14 (2)    14 (2)     3 (1) 
domain               |    8 (1)    10 (1)     0 (0) 
network              |    0 (0)     1 (0)     0 (0) 


### label set histogram

In [86]:
label_hist = {}

for pull_request in pull_requests:
    label = ", ".join(pull_request["codes"])
    label = label.replace("feature, feature", "feature")
    if label_hist.get(label, None) is None:
        label_hist[label] = 1
    else:
        label_hist[label] = label_hist[label] + 1
        

print(label_hist)

{'instance, saving': 25, 'awareness, storage': 10, 'alert': 28, 'awareness, feature': 6, 'awareness, instance': 10, 'saving, storage': 19, 'awareness, increase': 3, 'awareness, increase, storage': 3, 'saving': 10, 'awareness, saving, storage': 1, 'cluster, instance, saving': 4, 'networking, provider, saving': 1, 'awareness, feature, instance': 6, 'billing_mode, saving': 1, 'awareness, feature, increase': 3, 'billing_mode, feature, provider, storage': 1, 'feature, instance, saving': 14, 'increase, networking': 1, 'provider, saving': 2, 'billing_mode, storage': 4, 'awareness': 13, 'feature, saving, storage': 6, 'awareness, feature, storage': 2, 'provider, saving, storage': 1, 'awareness, feature, increase, storage': 3, 'networking, saving': 9, 'policy, saving, storage': 4, 'feature, policy, saving': 1, 'provider': 1, 'feature, instance, policy': 1, 'instance, networking, saving': 1, 'feature, instance, saving, storage': 2, 'awareness, cluster': 9, 'alert, feature': 2, 'area, awareness': 

### Knowledge graph

In [88]:
label_knowledge = {}

for pull_request in pull_requests:
    labels = pull_request["codes"]
    
    for label in labels:
    
        if label_knowledge.get(label, None) is None:
            label_knowledge[label] = {}
    
        for neighbour_label in labels:
            if neighbour_label != label:
                if label_knowledge[label].get(neighbour_label, None) is None:
                    label_knowledge[label][neighbour_label] = 1
                else:
                    label_knowledge[label][neighbour_label] = label_knowledge[label][neighbour_label] + 1


format_str = "{:12} |"
header = [""]

for label in label_knowledge:
    format_str += " {:12} "
    header.append(label)

print(format_str.format(*header))

for label in label_knowledge:
    row = [label]
    for neighbour in label_knowledge:
        if neighbour == label:
            row.append("")
        elif label_knowledge[label].get(neighbour, None) is not None:
            row.append(label_knowledge[label][neighbour])
        else:
            row.append(0)
    print(format_str.format(*row))


             | instance      saving        awareness     storage       alert         feature       increase      cluster       networking    provider      billing_mode  policy        area         
instance     |                         52            24             6             1            31             6             8             2             0             0             1             0 
saving       |           52                           3            36             1            37             2            16            13             4             2             5             0 
awareness    |           24             3                          22             1            22            15            11             5             1             2             0             3 
storage      |            6            36            22                           0            16             6             1             0             2             5             4             1 
alert        | 

### pull request - commit intersections

In [121]:
# first we need to combine pull requests with their respective commit hashes

commits_per_pr_file = open(STEP8_TF_KEYWORD_PR)
commits_per_pr = json.load(commits_per_pr_file)

prs = []
for commit_pr in commits_per_pr:

    # Get labeled commits 
    commits = []
    links = [commit_link["url"] for commit_link in commit_pr["pull_request"]["commits"]]
    for commit in all_commits:
        if commit["url"] in links:
            commits.append(commit)

    # Get labeled pr
    pr = None
    for pull in all_pull_requests:
        if pull["url"] == commit_pr["pull_request"]["url"]:
            pr = pull

    # turn commit labels into single ordered list of unique labels
    commit_codes = []
    for codes in [commit["codes"] for commit in commits]:
        commit_codes += codes
    commit_codes = list(set(commit_codes))
    commit_codes.sort()

    # turn pr labels into ordered list of labels
    pr_codes = pr["codes"]
    pr_codes.sort()

    if len(commit_codes) > 0:
        prs.append(
            {
                "url": pr["url"],
                "pr-labels": pr_codes,
                "commit-labels": commit_codes,
                "commits": len(commits)
            })

print("amount of identical labels for pr and commits:", len([pr for pr in prs if pr["pr-labels"] == pr["commit-labels"]]))
total_common = 0
total_pr_count = 0
total_commit_count = 0

for pr in prs:
    count = 0
    for label in pr["pr-labels"]:
        if label in pr["commit-labels"]:
            count += 1
    pr["common-labels"] = count
    total_common += count
    total_pr_count += len(pr["pr-labels"])
    total_commit_count += len(pr["commit-labels"])

print("pr labels:", total_pr_count, "common labels:", total_common, "commit labels:", total_commit_count)

with open(COMMIT_PR_INTERSECTION , "w") as outfile:
    json.dump(prs, outfile)

amount of identical labels for pr and commits: 25
pr labels: 305 common labels: 157 commit labels: 349
